In [36]:
# Import necessary libraries
import torch.cuda as torch_cuda
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
import matplotlib.pyplot as plt
import numpy as np

from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

In [37]:
print('===VERIFY GPU===')
print('CUDA IS AVAILABLE:', torch_cuda.is_available())
print('DEVICE COUNT:', torch_cuda.device_count())
print('DEVICE NAME:', torch_cuda.get_device_name(0))

# If CUDA is available, print GPU details
if torch.cuda.is_available():
    print(f"Number of GPUs: {torch_cuda.device_count()}")
    print(f"GPU Name: {torch_cuda.get_device_name(0)}")
    print(f"GPU Memory Allocated: {torch_cuda.memory_allocated(0)} bytes")
    print(f"GPU Memory Cached: {torch_cuda.memory_reserved(0)} bytes")

# Select device: GPU if available, otherwise CPU
device = torch.device('cuda:0' if torch_cuda.is_available() else 'cpu')

===VERIFY GPU===
CUDA IS AVAILABLE: True
DEVICE COUNT: 1
DEVICE NAME: NVIDIA RTX A4000
Number of GPUs: 1
GPU Name: NVIDIA RTX A4000
GPU Memory Allocated: 621207040 bytes
GPU Memory Cached: 807403520 bytes


## Dataset

In [33]:
# Define hyperparameters
batch_size = 32

# Define transformations: Convert to tensor and normalize
transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

mnist_trainset = datasets.MNIST(
    root='../Data',
    train=True,
    download=True,
    transform=transformations
)

mnist_testset = datasets.MNIST(
    root='../Data',
    train=False,
    download=True,
    transform=transformations
)

trainloader = DataLoader(
    mnist_trainset,
    batch_size=batch_size,
    shuffle=True
)

testloader = DataLoader(
    mnist_testset,
    batch_size=batch_size,
    shuffle=False,
)

## CNN

In [5]:
momentum_coeff = 0.9
learning_rate = 0.01
learning_decay = 10**-9
epochs = 100

# Classes for MNIST digits
classes = (0,1,2,3,4,5,6,7,8,9)

In [29]:
# Define a multi-layer perceptron (MLP) mode

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # First convolutional block
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)  
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1) 
        self.pool1 = nn.MaxPool2d(kernel_size=2) 
        self.dropout1 = nn.Dropout(0.25) 

        # Second convolutional block
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)  
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)  
        self.pool2 = nn.MaxPool2d(kernel_size=2)  
        self.dropout2 = nn.Dropout(0.25)  

        # Fully connected layers
        self.fc1 = nn.Linear(3136, 3136)  
        self.dropout3 = nn.Dropout(0.5) 
        self.fc2 = nn.Linear(3136, 512)
        self.fc3 = nn.Linear(512, 10) 

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        x = self.dropout1(x)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        x = self.dropout2(x)

        x = x.view(-1, 64 * 7 * 7)

        x = F.relu(self.fc1(x))
        x = self.dropout3(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x) 
        return x


# Instantiate and move model to the selected device
cnn = CNN().to(device)

loss_function = nn.CrossEntropyLoss()  # CrossEntropyLoss expects raw logits
optimizer = optim.SGD(cnn.parameters(), lr=learning_rate,
                      momentum=momentum_coeff, weight_decay=learning_decay)


def train(epoch):

    cnn.train()
    for batch_idx, (data, target) in enumerate(trainloader):

        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        output = cnn(data)

        loss = loss_function(output, target)

        loss.backward()

        optimizer.step()

        if batch_idx % 25 == 0:
            print('Train Epoch: {:>3}\t[{:>5}/{:>5} ({:>3.0f}%)]\tLoss: {:>7.6f}'.format(
                epoch + 1, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))


def test():
    cnn.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in testloader:

            data, target = data.to(device), target.to(device)

            output = cnn(data)

            test_loss += loss_function(output, target).item()

            pred = output.argmax(dim=1, keepdim=True)

            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(testloader)

    print('\nTest set: Average loss: {:>10.4f}, Accuracy: {:>5}/{:>5} ({:>3.0f}%)\n'.format(
        test_loss, correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


for epoch in range(epochs):
    train(epoch)
    test()


Train Epoch:   1	[    0/60000 (  0%)]	Loss: 2.301846
Train Epoch:   1	[  800/60000 (  1%)]	Loss: 2.291595
Train Epoch:   1	[ 1600/60000 (  3%)]	Loss: 2.283168
Train Epoch:   1	[ 2400/60000 (  4%)]	Loss: 2.271148
Train Epoch:   1	[ 3200/60000 (  5%)]	Loss: 1.465142
Train Epoch:   1	[ 4000/60000 (  7%)]	Loss: 0.618756
Train Epoch:   1	[ 4800/60000 (  8%)]	Loss: 0.695973
Train Epoch:   1	[ 5600/60000 (  9%)]	Loss: 0.757515
Train Epoch:   1	[ 6400/60000 ( 11%)]	Loss: 0.718815
Train Epoch:   1	[ 7200/60000 ( 12%)]	Loss: 0.325582
Train Epoch:   1	[ 8000/60000 ( 13%)]	Loss: 0.167876
Train Epoch:   1	[ 8800/60000 ( 15%)]	Loss: 0.174673
Train Epoch:   1	[ 9600/60000 ( 16%)]	Loss: 0.234469
Train Epoch:   1	[10400/60000 ( 17%)]	Loss: 0.217655
Train Epoch:   1	[11200/60000 ( 19%)]	Loss: 0.323030
Train Epoch:   1	[12000/60000 ( 20%)]	Loss: 0.155021
Train Epoch:   1	[12800/60000 ( 21%)]	Loss: 0.112612
Train Epoch:   1	[13600/60000 ( 23%)]	Loss: 0.268494
Train Epoch:   1	[14400/60000 ( 24%)]	Loss: 0.

In [34]:
torch.save(cnn.state_dict(), '../Saved/MNIST_CNN_TRAINED.pt')